In [12]:
import yfinance as yf
import numpy as np
import pandas as pd
from get_data_yf import get_symbols
import os

#TODO Change filepaths using os.pathjoint to avoid os dependency if you find time

RECOMMENDATION_WEIGHTS = [2, 1, 0, -1, -2] #weights for strongBuy, buy, hold, sell,	strongSell advisors respectively
TIMEZONE_WEIGHTS = [4, 3, 2, 1] # weights for 0, -1, -2, -3rd months respectively
ADVICE_COUNT = 'max' #for all data --> 'max'                    #how many of best scored symbols we want to get data
folder = 'data/recommendations'
recommendation_data_folder = 'data/recommendation_data'

score_filename = 'scores.xlsx'
recommendations_filename = 'recommendations.xlsx'

In [ ]:
def create_weigth_kernel(vertical_weights, horizontal_weights):
    kernel_values = np.outer(vertical_weights, horizontal_weights)
    kernel = pd.DataFrame(kernel_values)
    kernel.rename(columns={0:'strongBuy', 1:'buy', 2:'hold', 
                              3:'sell', 4:'strongSell'}, inplace = True)
    
    return kernel

In [ ]:
def recommendation_score(recommendations, weight_kernel):
    #returns score and how many anaylst we have as advisor in total
    sum = recommendations.to_numpy().sum()
    if sum == 0:
        return None, None
    weighted_sum = (weight_kernel * recommendations).to_numpy().sum()
    return weighted_sum / sum, sum 

In [ ]:
def evaluate_symbols(symbols, timezone_weights, recommendation_weights):
    #Will return a df that stores scores and advisor count for a symbol
    scores = {}
    weight_kernel = create_weigth_kernel(timezone_weights, recommendation_weights)
    
    for symbol in symbols:
        recommendations = yf.Ticker(symbol).recommendations
        if  recommendations.empty:
            print(f"Couldn't fetch data for {symbol}.")
            continue
        
        recommendations = recommendations.drop('period', axis='columns')
        score, advisor = recommendation_score(recommendations, weight_kernel)
        if score:  
            scores[symbol] = [score, advisor]
        else: 
            print(f"No advisors found for {symbol}.") 
            
    scores_df = pd.DataFrame.from_dict(scores, orient='index', columns=['Scores', 'Advisors'])
    scores_df = scores_df.sort_values(by='Scores', ascending=False)
    
    return scores_df 

In [ ]:
def evaluate_files(recommendation_data_folder, timezone_weights, recommendation_weights):
    scores = {}
    weight_kernel = create_weigth_kernel(timezone_weights, recommendation_weights)
    
    files = os.listdir(recommendation_data_folder)
    for filename in files:
        if filename.endswith('.csv'):
            recommendations = pd.read_csv(f'{recommendation_data_folder}/{filename}')
            recommendations = recommendations.drop('period', axis='columns')
            
            score, advisor = recommendation_score(recommendations, weight_kernel)
            symbol = os.path.splitext(filename)[0]
            
            if score:  
                scores[symbol] = [score, advisor]
            else: 
                print(f"No advisors found for {symbol}.")
    
    scores_df = pd.DataFrame.from_dict(scores, orient='index', columns=['Scores', 'Advisors'])
    scores_df = scores_df.sort_values(by='Scores', ascending=False)
    
    return scores_df 

In [20]:
def write_recommendations(scores, folder, score_filename, recommendations_filename, advice_count=10):
    if advice_count == 'max':
        advice_count = len(scores) 
    
    with pd.ExcelWriter(f'{folder}/{score_filename}', engine='xlsxwriter') as writer:
        scores.to_excel(writer)
        print("Score file saved.")
        
    filenames = os.listdir(recommendation_data_folder)  
    wanted_symbols = scores.head(advice_count).index.tolist()
    
    with pd.ExcelWriter(f'{folder}/{recommendations_filename}', engine='xlsxwriter') as writer:
        starting_row = 0
        for symbol in wanted_symbols:
            
            symbol_filename = f'{symbol}.csv'
            if symbol_filename in filenames:
                recommendation = pd.read_csv(f'{recommendation_data_folder}/{symbol_filename}')  
            else:
                recommendation = yf.Ticker(symbol).recommendations
                
            if not recommendation.empty:
                recommendation.to_excel(writer, startrow=starting_row)
                writer.sheets['Sheet1'].write(starting_row, 0, symbol)
                starting_row += len(recommendation) + 3
            else: 
                print(f"Couldn't write the recommendations for {symbol}.")
        print("Recommendation file saved.")

In [17]:
def main_wo_data():
    #This is the main function to get files without recommendation dfs being 
    #present at the moment, it calls API for every symbol, so if we have too big
    #symbol list, it may trigger API limit. Use main_w_data if you have 
    #recommendation files in your hand
    symbols = get_symbols() #Change this with get_symbols() when you want real -big- set
    scores = evaluate_symbols(symbols, TIMEZONE_WEIGHTS, RECOMMENDATION_WEIGHTS)
    write_recommendations(scores, folder, score_filename, recommendations_filename, ADVICE_COUNT)

In [21]:
def main_w_data():
    scores = evaluate_files(recommendation_data_folder, TIMEZONE_WEIGHTS, RECOMMENDATION_WEIGHTS)
    write_recommendations(scores, folder, score_filename, recommendations_filename, ADVICE_COUNT)

In [23]:
if __name__ == '__main__':
    main_w_data()

No advisors found for AAME.
No advisors found for AAU.
No advisors found for ABVC.
No advisors found for ACV.
No advisors found for ADD.
No advisors found for ADX.
No advisors found for AEHL.
No advisors found for AEI.
No advisors found for AFB.
No advisors found for AGAE.
No advisors found for AGD.
No advisors found for AHG.
No advisors found for AIH.
No advisors found for AIMD.
No advisors found for AIRT.
No advisors found for AIU.
No advisors found for AIXI.
No advisors found for AKO-A.
No advisors found for ALLR.
No advisors found for ALRS.
No advisors found for ALTI.
No advisors found for ALUR.
No advisors found for ALYAF.
No advisors found for AMBO.
No advisors found for AMPE.
No advisors found for ANRO.
No advisors found for ANTE.
No advisors found for AOD.
No advisors found for APWC.
No advisors found for ARKR.
No advisors found for ARL.
No advisors found for AROW.
No advisors found for ARTW.
No advisors found for AS.
No advisors found for ASA.
No advisors found for ASG.
No adv